In [2]:
# default_exp load_data

In [4]:
# export

import seaborn as sn
import pandas as pd
import json
import logging
from path import Path
from datetime import datetime
from dateutil.relativedelta import relativedelta
from mlfinlab.data_structures import get_dollar_bars, get_tick_bars, get_time_bars, get_volume_bars

from mlbt.utils import NumpyEncoder

# You'll likely have to change these if you're intending to run the code yourself
# TODO: Factor out into settings.py file
DATA_DIR = Path("~/Dropbox/algotrading/data").expanduser()
F_PAYLOAD_DIR = Path("~/pr/fincl/frontend/public/payloads").expanduser()
DAILY_DATA_DIR = DATA_DIR / "daily"

SYMBOLS_CSV = pd.read_csv(DATA_DIR / "symbols.csv", index_col="iqsymbol")

LENGTH_RANKING = ['@SP#C','@W#C','@BO#C','@QM#C','@QG#C','EZ#C','@KC#C','@C#C','XG#C','@S#C','EX#C','@SB#C','@OJ#C','@NKD#C','@EMD#C','BD#C','LF#C','QNG#C','QCL#C','@HE#C','@CC#C','@CT#C','@LE#C','@ES#C','@TY#C','QSI#C','QPL#C','@FV#C','@O#C','LG#C','@SM#C','@GF#C','QGC#C','@YM#C','@TU#C','QPA#C','@UB#C','@NQ#C','@ED#C','GAS#C','@MME#C','QHG#C','QHO#C','@BP#C','@CD#C','@RP#C','@NE#C','@AD#C']

/Users/doda/anaconda3/envs/metal/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/doda/anaconda3/envs/metal/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [1]:
# export

def get_symbols(symbol_groups):
    lists = {"us_index": ["@NQ#C", "@ES#C", "@YM#C"]}
    if len(symbol_groups) == 1 and symbol_groups[0] in lists:
        return lists[symbol_groups[0]]

    sectors = {
        "agriculture": "Agriculture",
        "currency": "Currency",
        "energy": "Energy",
        "equity_index": "Equity Index",
        "interest_rate": "Interest Rate",
        "metals": "Metals",
    }
    symbol_groups = [sectors[x] for x in symbol_groups]
    picked = SYMBOLS_CSV[SYMBOLS_CSV["Sector"].isin(symbol_groups)]

    ignore = ["@LH#C"]
    return [x for x in picked.index.values if x not in ignore]


def load_contract(contract_name, directory):
    series = pd.read_csv(
        DATA_DIR / directory / "{}.csv".format(contract_name), index_col=0
    )
    series = series[::-1]
    if directory == "minutely":
        series["Time"] = series["date"] + " " + series["time"]
        series = series.set_index(
            pd.to_datetime(series["Time"], format="%Y-%m-%d 0 days %H:%M:00.000000000")
        )
    else:
        series["Time"] = series["date"]
        series = series.set_index(pd.to_datetime(series["Time"], format="%Y-%m-%d"))

    series = series[["open_p", "close_p", "prd_vlm", "Time"]]
    series = series.rename(
        columns={"close_p": "Close", "open_p": "Open", "prd_vlm": "Volume"}
    )
    series["Instrument"] = contract_name
    return series


def load_contracts(symbol, directory="minutely", start_date=None, end_date=None):
    contract_names = [
        x.basename().namebase
        for x in (DATA_DIR / directory).files("*{}*".format(symbol))
    ]
    loaded = [load_contract(x, directory) for x in contract_names]
    loaded = list(sorted(loaded, key=lambda x: x.index[-1]))
    first = loaded[0]
    # cut out from later contracts what former contracts already have
    zipped = zip(loaded, loaded[1:])
    cut_contracts = [
        latter.truncate(before=former.index[-1] + pd.Timedelta(minutes=1))
        for former, latter in zipped
    ]

    concatted = pd.concat([first] + cut_contracts)
    return concatted.truncate(before=start_date, after=end_date)


def load_all_cont_contracts():
    all_continuous_contracts = DAILY_DATA_DIR.files("*#C*")
    all_continuous_contracts = [x.basename().namebase for x in all_continuous_contracts]
    return {name: load_contract(name, "daily") for name in all_continuous_contracts}


def get_data(symbol, frequency, start_date, end_date):
    # Include up to 1 year prior for feature engineering
    # we blindly assume no code wants a longer warm-up period than that
    return load_contracts(
        symbol,
        frequency,
        start_date - relativedelta(years=1) if start_date else None,
        end_date,
    )

def process_bars(bars, size, type_, resolution="MIN"):
    # Renaming our bar columns & format for mlfinlab for processing and then back into our original format
    # OHL from 1-min bars are ignored
    fun = {
        "time": get_time_bars,
        "tick": get_tick_bars,
        "volume": get_volume_bars,
        "dollar": get_dollar_bars,
    }[type_]

    bars = bars[['Close', 'Volume']].reset_index()
    bars.columns = ['date_time', 'close', 'volume']
    if type_ == "time":
        s_bars = fun(bars, resolution=resolution, num_units=size)
    else:
        s_bars = fun(bars, threshold=size)
    bars = s_bars[['date_time', 'open', 'high', 'low', 'close', 'volume', 'cum_dollar_value', 'cum_ticks', 'cum_buy_volume']]
    bars.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dollar Volume', 'Num Ticks', 'Buy Volume']
    if type_ == "time":
        bars = bars.set_index(s_bars['date_time'].apply(datetime.fromtimestamp).values)
    else:
        bars = bars.set_index('Time', drop=False)
    return bars


def load_and_sample_bars(symbol, start_date, end_date, type_, size=None):
    bars = get_data(symbol, "minutely", start_date, end_date)
    bars["Dollar Volume"] = bars["Volume"] * bars["Close"]

    if size is None:
        size = determine_bar_size(bars, type_)

    return process_bars(bars, size, type_), size


In [2]:
# export
def determine_bar_size(bars, bar_type):
    # Return bar size to have approx. 25 bars per day for the year 2019
    col = {"dollar": "Dollar Volume", "volume": "Volume"}[bar_type]
    bar_size = bars[bars.index.year == 2019][col].sum() / 252 / 25
    return bar_size

In [3]:
# export 

def safe_feat_name(feat_c, safe_for_fs=True):
    """Turn our {feature:dict} into a pythonic function_invocation(like=string) for display & file-path purposes"""
    feat_c = feat_c.copy()
    for k,v in feat_c.items():
        if isinstance(v, dict):
            feat_c[k] = safe_feat_name(v, safe_for_fs)

    if safe_for_fs:
        name = feat_c.pop("name")
        dumped = json.dumps(
            feat_c,
            sort_keys=True,
            separators=(',', '=')
        ).replace('"', '').replace('{', '(').replace('}', ')')
        return f"{name}{dumped}"
    else:
        return json.dumps(
            feat_c,
            sort_keys=True,
        )

def load_hdf(path):
    try:
        if path.exists() and path.size:
            return pd.read_hdf(path, 'table')
    except:
        logging.exception("could not load_hdf")
        # Does this fix our weird error?
        with open(path) as f:
            pass


def save_hdf(obj, path):
    obj.to_hdf(path, 'table')
    return path


def bars_path(symbol, c):
    return DATA_DIR / c['bar_type'] / f"{symbol}_bars.h5"


def events_b_path(symbol, c):
    return DATA_DIR / c['bar_type'] / f"{symbol}_events_{c['vol_estimate']}_{c['binarize']}_{c['binarize_params']}_{c['downsampling']}.h5"


def feats_path(symbol, c):
    feat_names = '-'.join(sorted(set(x.split('_')[0] for x in c['features'])))
    return DATA_DIR / c['bar_type'] / f"{symbol}_feats_{feat_names}.h5"


def feat_path(c, feat_c):
    # Make a compact, unique path for this feature config
    basename = safe_feat_name(feat_c)
    return DATA_DIR / 'features' / c['bar_type'] / f"{basename}.h5"

def find_params(d, s=None):
    s = s or set()
    for k, v in d.items():
        if isinstance(v, dict):
            return find_params(v, s)
        else:
            s.add(str(v))
        s.add(str(k))
    return s

def imp_path(symbol, c):
    feat_names = set.union(*[find_params(feat) for feat in c['features']])
    feat_names = ''.join([x[:2] for x in sorted(feat_names - {'name', 'window', 'stdev', 'lag'})])
    return DATA_DIR / c['bar_type'] / f"{symbol}_fimp_{c['binarize']}_{c['binarize_params']}_{c['alpha']}_{c['alpha_params']}_{feat_names}_{c['feat_imp_method']}.h5"


def payload_path(symbols, c):
    symbols_s = '-'.join(c['symbol_groups'] or c['symbols'])
    return DATA_DIR / 'payloads' / f"payload_{symbols_s}_{c['bar_type']}_{c['binarize']}_{c['binarize_params']}_{c['alpha']}_{c['alpha_params']}_{c['classifier']}.json"

###


def load_bars(symbol, config):
    if config["load_from_disk"]:
        path = bars_path(symbol, config)
        return load_hdf(path)


def save_bars(symbol, config, bars):
    if config["save_to_disk"]:
        path = bars_path(symbol, config)
        return save_hdf(bars, path)


def load_events_b(symbol, config):
    if config["load_from_disk"]:
        path = events_b_path(symbol, config)
        return load_hdf(path)


def save_events_b(symbol, config, events_b):
    if config["save_to_disk"]:
        path = events_b_path(symbol, config)
        return save_hdf(events_b, path)


def load_feat(config, feat_config):
    if config["load_from_disk"]:
        path = feat_path(config, feat_config)
        return load_hdf(path)


def save_feat(config, feat_config, feat):
    if config["save_to_disk"]:
        path = feat_path(config, feat_config)
        return save_hdf(feat, path)


def load_imp(symbol, config):
    if config["load_from_disk"]:
        path = imp_path(symbol, config)
        return load_hdf(path)


def save_imp(symbol, config, imp):
    if config["save_to_disk"]:
        path = imp_path(symbol, config)
        return save_hdf(imp, path)


def load_payload(symbols, config):
    if config["load_from_disk"]:
        path = payload_path(symbols, config)
        try:
            if path.exists() and path.size:
                with open(path) as f:
                    return json.load(f)
        except:
            logging.error(f"corrupted payload: {path}")


def save_payload(symbols, config, payload):
    if config["save_to_disk"]:
        path = payload_path(symbols, config)
        with open(path, 'w') as f:
            json.dump(payload, f, cls=NumpyEncoder)
        return path


In [44]:
feat1 = {'name':'a', 'b':2, 'c':3, 'd':{'name': 'dede', 'bb':33}}
feat2=   {'name': 'auto',
    'window': 250,
    'lag': 10,
    'symbol': {'name': 'stdev',
     'window': 250,
     'symbol': {'name': 'ffd', 'd': 0.5}}}

safe_feat_name(feat)

'auto(lag=10,symbol=stdev(symbol=ffd(d=0.5),window=250),window=250)'

In [45]:
imp_path('arst', {'features': [feat1, feat2], 'bar_type': 'derp', 'binarize': 'yes', 'binarize_params': 'yes', 'alpha': 'no', 'alpha_params': 'no', 'feat_imp_method': 'ya'})

Path('C:\\Users\\doda/Dropbox/algotrading/data\\derp\\arst_fimp_yes_yes_no_no_0.5-10-2-250-3-33-a-auto-b-bb-c-d-dede-ffd_ya.h5')